# Core Concepts Illustrated Through a Simple Game

This section of the user guide explains the core concepts of the library through playing a simple game. It also has references to other sections for more advanced use cases.

## Defining the Game

The first thing you might want to do is to define the game itself. By default, the {class}`~maverick.game.Game` class sets up a No-Limit Texas Hold'em game for you.

```{note}
You can read about how to set up other poker variants with custom rulesets {doc}`here <games>`.
```

In [1]:
from maverick import Game

game = Game(small_blind=10, big_blind=20, max_hands=2)

```{note}
Here we defined the maximum number of hands as 2, but this is optional. With that being said, an upper limit on the number of hands always exists, even if you dont define it. Refer to the {doc}`API Reference <../api_reference>` to see the default values for the input parameters.
```

```{tip}
It is recommended to always set a reasonable upper limit on the number of hands, especially if you use bots such as {class}`~maverick.players.foldbot.FoldBot` (as we do in the next section of this document).
```

The {class}`~maverick.game.Game` class is implemented as a state machine, which is a common design pattern. It has an attribute called `state`, which is an instance of {class}`~maverick.state.GameState`. The game maintains this state through events and state transitions.

In [2]:
game.state.state_type

<GameStateType.WAITING_FOR_PLAYERS: 1>

The which is an instance of {class}`~maverick.state.GameState` class inherited from PyDantic's {class}`~pydantic.main.BaseModel` class, hence it can be dumped to a dictionary:

In [3]:
game.state.model_dump()

{'state_type': <GameStateType.WAITING_FOR_PLAYERS: 1>,
 'street': <Street.PRE_FLOP: 0>,
 'players': [],
 'active_players': [],
 'current_player_index': 0,
 'deck': None,
 'community_cards': [],
 'pot': 0,
 'current_bet': 0,
 'min_bet': 0,
 'last_raise_size': 0,
 'small_blind': 10,
 'big_blind': 20,
 'ante': 0,
 'hand_number': 0,
 'button_position': 0}

We will issue the print statement above to illustrate how the state of the game changes.

## Defining the Players of the Game

The library contains built-in players you can use as training opponents. For an exhaustive list of available built-in players, refer to {ref}`this <players_api_reference>` section of the API Reference. What you have to tell at the minimum when setting up a player is the player's `name`, and the amount of chips they start the game with. Among other things that might change during a game, the amount of chips (aka. stack) is part of the player's state.

In [4]:
from maverick.players import FoldBot, CallBot, AggressiveBot
from maverick import PlayerLike, PlayerState


players: list[PlayerLike] = [
    CallBot(name="CallBot", state=PlayerState(stack=1000)),
    AggressiveBot(name="AggroBot", state=PlayerState(stack=1000)),
    FoldBot(name="FoldBot", state=PlayerState(stack=1000)),
]

for player in players:
    game.add_player(player)

```{note}
You can read about how to implement custom players {doc}`here <custom_players>`.
```

Every implemented player class must adhere to the protocol defined by the {class}`~maverick.protocol.PlayerLike` class. Every instance of a player class has a `name`, and `id`, and a `state`. The `state` attribute is an instance of {class}`~maverick.playerstate.PlayerState` and it captures all information of a player that might change during a game

In [5]:
game.state.model_dump()

{'state_type': <GameStateType.READY: 2>,
 'street': <Street.PRE_FLOP: 0>,
 'players': [{'id': '40c2cf60cf2a4b95a19657ba14e9d099',
   'name': 'CallBot',
   'state': {'seat': 0,
    'state_type': <PlayerStateType.ACTIVE: 1>,
    'stack': 1000,
    'holding': None,
    'current_bet': 0,
    'total_contributed': 0,
    'acted_this_street': False},
   '__class__.__name__': 'CallBot'},
  {'id': '5552cb355e1446f2ba6ed20658fdc837',
   'name': 'AggroBot',
   'state': {'seat': 1,
    'state_type': <PlayerStateType.ACTIVE: 1>,
    'stack': 1000,
    'holding': None,
    'current_bet': 0,
    'total_contributed': 0,
    'acted_this_street': False},
   '__class__.__name__': 'AggressiveBot'},
  {'id': '4960b9cda5d04aa2817901053042b95b',
   'name': 'FoldBot',
   'state': {'seat': 2,
    'state_type': <PlayerStateType.ACTIVE: 1>,
    'stack': 1000,
    'holding': None,
    'current_bet': 0,
    'total_contributed': 0,
    'acted_this_street': False},
   '__class__.__name__': 'FoldBot'}],
 'active_play

## Playing the Game

The following cell kicks off the event flow of the game and finishes when the game has terminated. A game might terminate for a number of reasons:

- there are not enough players to continue (because everyone got eliminated except one)
- the number of hands reached the maximum number of hands defined at instantiation (current case)

Before you start a game, you might want to configure logging, so you can inspect the event flow of the game without distractions.

In [6]:
import logging

# Configure logging such that we only get the log messages of the game
logging.basicConfig(level=logging.INFO, format="%(name)s: %(message)s", force=True)
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("maverick").setLevel(logging.INFO)

And now we can kick off the game.

In [7]:
# Start the game
game.start()

maverick: Game started.

maverick: ============================== Hand 1 ==============================

maverick: PRE_FLOP | Dealing hole cards. Button: CallBot
maverick: PRE_FLOP | Posting small blind of 10 by player AggroBot. Remaining stack: 990
maverick: PRE_FLOP | Posting big blind of 20 by player FoldBot. Remaining stack: 980
maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 980.
maverick: PRE_FLOP | Current pot: 50 | Current bet: 20
maverick: PRE_FLOP | Player AggroBot raises by 30 chips to total bet 40. Remaining stack: 960.
maverick: PRE_FLOP | Current pot: 80 | Current bet: 40
maverick: PRE_FLOP | Player FoldBot folds.
maverick: PRE_FLOP | Current pot: 80 | Current bet: 40
maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 960.
maverick: PRE_FLOP | Current pot: 100 | Current bet: 40
maverick: PRE_FLOP | Betting round complete

maverick: FLOP | Dealt flop. Community cards: ['4♠', '6♦', 'A♥']
maverick: FLOP | Player AggroBot bet

In [8]:
game.state.model_dump()

{'state_type': <GameStateType.GAME_OVER: 11>,
 'street': <Street.SHOWDOWN: 4>,
 'players': [{'id': '40c2cf60cf2a4b95a19657ba14e9d099',
   'name': 'CallBot',
   'state': {'seat': 0,
    'state_type': <PlayerStateType.ACTIVE: 1>,
    'stack': 680,
    'holding': {'cards': [{'suit': <Suit.SPADES: 'S'>,
       'rank': <Rank.SEVEN: 7>},
      {'suit': <Suit.HEARTS: 'H'>, 'rank': <Rank.TEN: 10>}]},
    'current_bet': 0,
    'total_contributed': 160,
    'acted_this_street': False},
   '__class__.__name__': 'CallBot'},
  {'id': '5552cb355e1446f2ba6ed20658fdc837',
   'name': 'AggroBot',
   'state': {'seat': 1,
    'state_type': <PlayerStateType.ACTIVE: 1>,
    'stack': 1350,
    'holding': {'cards': [{'suit': <Suit.SPADES: 'S'>,
       'rank': <Rank.JACK: 11>},
      {'suit': <Suit.CLUBS: 'C'>, 'rank': <Rank.TEN: 10>}]},
    'current_bet': 0,
    'total_contributed': 160,
    'acted_this_street': False},
   '__class__.__name__': 'AggressiveBot'},
  {'id': '4960b9cda5d04aa2817901053042b95b',
  

### How does it work?

When you call `game.start()`, an event is added to the event queue of the game. When an event is added to the event queue, it usually results in a state transition and triggers downstream events and the cycle continues until the event queue is fully drained. Certain events require players to take action, and the players respond with an action. You will learn about these when you implement your first custom player here: {doc}`here <custom_players>`.

## Inspecting the Results

Every player has a state attribute, which is an instance of {class}`~maverick.playerstate.PlayerState`. You can use this class to print the actual stack of every player.

In [9]:
for player in players:
    print(f"{player.name} - Stack: {player.state.stack}")

CallBot - Stack: 680
AggroBot - Stack: 1350
FoldBot - Stack: 970


## Inspecting the Event History

Games maintain a full history of events and it's accessible through the `history` property of an instance. It returns a dictionary with keys 'game', 'hand' and 'street', all mapping to a list of {class}`~maverick.events.GameEvent`. The three keys map to the full events and the events of the current hand and street respectively. The latter two are subsets of the first one and they are probably only interesting during the game.

In [ ]:
# Show the first 10 events that occurred in the game
game.history[:10]

[GameEvent(id='724c9585250049a592696505620dcd7f', ts=1768862898.358909, type=<GameEventType.PLAYER_JOINED: 14>, hand_number=0, street=<Street.PRE_FLOP: 0>, player_id='40c2cf60cf2a4b95a19657ba14e9d099', action=None, payload={}),
 GameEvent(id='6bc3e0726c534770bc6b4ab0848859f6', ts=1768862898.358966, type=<GameEventType.PLAYER_JOINED: 14>, hand_number=0, street=<Street.PRE_FLOP: 0>, player_id='5552cb355e1446f2ba6ed20658fdc837', action=None, payload={}),
 GameEvent(id='b886429c4bde40eb8c5a51b62b14215e', ts=1768862898.3589828, type=<GameEventType.PLAYER_JOINED: 14>, hand_number=0, street=<Street.PRE_FLOP: 0>, player_id='4960b9cda5d04aa2817901053042b95b', action=None, payload={}),
 GameEvent(id='19e5d96c74464421973c33ec6366ed54', ts=1768862898.372385, type=<GameEventType.GAME_STARTED: 1>, hand_number=0, street=<Street.PRE_FLOP: 0>, player_id=None, action=None, payload={}),
 GameEvent(id='ffb320c55bf54cf7a72793c238133068', ts=1768862898.372874, type=<GameEventType.HAND_STARTED: 2>, hand_numb

If you are only interested in the full game history, you can also use the `game_history` attribute of the instance and the following block is equivalent to the previous one.